## Data preparation of four scenarios for the full alignment case

In [3]:
import numpy as np
import librosa as lb
import soundfile as sf
from pathlib import Path
import matplotlib.pyplot as plt

Before we generate 4 different scenarios, we first define a function `splice_audio`.

In [4]:
def splice_audio(audio, start_sec, end_sec, sr):
    return audio[int(start_sec * sr): int(end_sec * sr)]

Define the pathway

In [5]:
op17no4_1981_path = Path('/Users/huazeliu/Desktop/E207PROJECT/train_data/Chopin_Op017No4_Ashkenazy-1981_pid9058-13.wav')
op17no4_1995_path = Path('/Users/huazeliu/Desktop/E207PROJECT/train_data/Chopin_Op017No4_Block-1995_pid9064-04.wav')
op63no3_path = Path('/Users/huazeliu/Desktop/E207PROJECT/train_data/Chopin_Op063No3_Morozova-2008_pid610633-04.wav')


Load the audio file with `librosa`

In [6]:
op17no4_1981, sr_1 = lb.load(op17no4_1981_path)
op17no4_1995, sr_2 = lb.load(op17no4_1995_path)
op63no3_2008, sr_3 = lb.load(op63no3_path)

AttributeError: module 'numba' has no attribute 'core'

### Scenario I: Matching
- In this case, we use `Op17No4_1981` and `Op17No4_1995`
- In this case, we align two complete recordings of the same Mazurka

if the sampling rates of Op17_1981 and Op17_1995 are not the same

In [6]:
if sr_1 != sr_2:
    op17no4_1995 = lb.resample(op17no4_1995, orig_sr=sr_2, target_sr=sr_1)

In [8]:
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_1/full_align_matching_recording1.wav', op17no4_1981, sr_1)
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_1/full_align_matching_recording2.wav', op17no4_1995, sr_1)

### Scenario II: Non-matching
- In this case, we use `Op17No4_1981` and `Op63No3_2008`
- In this case, we align two complete recordings of different Mazurkas

if the sampling rates of Op17_1981 and Op17_1995 are not the same

In [9]:
if sr_1 != sr_3:
    op63no3_2008 = librosa.resample(op63no3_2008, orig_sr=sr_3, target_sr=sr_1)

In [11]:
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_2/full_align_non_matching_recording1.wav', op17no4_1981, sr_1)
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_2/full_align_non_matching_recording2.wav', op63no3_2008, sr_1)

### Scenario III: Matching with a mismatching region in the middle
- In this case, we use `Op17No4_1981`, `Op17No4_1995` and `Op63No3_2008`
- Recording A: `Op17No4_1981`
- Recording B: `Op17No4_1995`, where a 30s chunk in the middle is spliced out and replaced with a 30s chunk from `Op63No3_2008`

In [12]:
def add_chunk(chunk_sec, main_audio, chunk_audio, sr=22050):
    half_len_main = len(main_audio)//2
    half_len_chunk = len(chunk_audio)//2

    # Calculate the start sample of the 30s chunk in the middle of main audio & chunk audio
    start_main = half_len_main - (chunk_sec * sr // 2)
    start_chunk = half_len_chunk - (chunk_sec * sr // 2)

    # Splice out the 30s chunk from main audio and replace with a chunk from chunk audio
    samples = chunk_sec * sr
    replacement = chunk_audio[start_chunk : start_chunk+samples]
    recording_b = np.concatenate([
        main_audio[:start_main],
        replacement,
        main_audio[start_main+samples:]
    ])

    return recording_b

Compute the Recording B

In [13]:
recording_b = add_chunk(30, op17no4_1995, op63no3_2008)

In [14]:
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_3/full_align_M_Mid_misM_recordingA.wav', op17no4_1981, sr_1)
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_3/full_align_M_Mid_misM_recordingB.wav', recording_b, sr_2)

### Scenario IV: Non-Matching with a matching region in the middle
- In this case, we use `Op17No4_1981`, `Op17No4_1995` and `Op63No3_2008`
- Recording A: `Op17No4_1981`
- Recording B: `Op63No3_2008`, where a 30s chunk in the middle is spliced out and replaced with a 30s chunk from `Op17No4_1995`

In [15]:
recording_b = add_chunk(30, op63no3_2008, op17no4_1995)

In [17]:
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_4/full_align_NM_Mid_M_recordingA.wav', op17no4_1981, sr_1)
sf.write('/Users/huazeliu/Desktop/E207PROJECT/Full_Alignment_Recordings/Scenario_4/full_align_NM_Mid_M_recordingB.wav', recording_b, sr_2)

In [29]:
# plt.figure(figsize=(12, 6))
# plt.plot(recording_b)
# plt.show()

In [28]:
# plt.figure(figsize=(12, 6))
# plt.plot(op17no4_1981)
# plt.show()